# 引用

In [74]:
import pandas as pd
import os
import shutil
from typing import List
import pandas as pd
from openpyxl import load_workbook

In [75]:
# 加载CSV文件
file_path = "esg-daip-core-web.csv"
df = pd.read_csv(file_path, encoding='utf-8')

In [76]:
folder_name = os.path.splitext(file_path)[0]
if not os.path.exists(folder_name):
    os.makedirs(folder_name, exist_ok=True)

filename = os.path.join(folder_name, os.path.basename("out.xlsx"))

def add_df_to_excel(df, sheet_name, filename=filename):
    # 加载现有的 excel 文件
    book = load_workbook(filename)

    # 创建 ExcelWriter 对象，使用这个 book
    writer = pd.ExcelWriter(filename, engine='openpyxl') 

    # 将这个 book 作为 writer 对象的书
    writer.book = book

    # 写入新的 dataframe 到新的工作表
    df.to_excel(writer, sheet_name, index=False)

    # 保存更改
    writer.save()
    writer.close()

In [77]:
# 基于'file'和'author'列来分组数据，并计算每个作者在每个文件的提交次数
author_file_commit_counts = df.groupby(['file', 'author']).size().reset_index(name='commit_count_per_author')

# 计算每个文件的总提交次数
total_commit_counts = df.groupby('file').size().reset_index(name='total_commit_count')

# 计算每个文件的总提交人数
total_author_counts = df.groupby('file')['author'].nunique().reset_index(name='total_author_count')

# 根据作者分组，计算每个作者的提交次数
author_commit_counts = df.groupby('author').size().reset_index(name='author_commit_count')

# 将这四个数据框合并
merged_df = pd.merge(author_file_commit_counts, total_commit_counts, on='file')
merged_df = pd.merge(merged_df, total_author_counts, on='file')
merged_df = pd.merge(merged_df, author_commit_counts, on='author')

# 计算每个作者的提交次数占总次数的百分比
merged_df['author_commit_percentage'] = (merged_df['commit_count_per_author'] / merged_df['total_commit_count']) * 100

# 计算每个作者占总提交人数的百分比
merged_df['author_total_percentage'] = (merged_df['author_commit_count'] / merged_df['total_author_count']) * 100

# 计算每个作者在每个文件的贡献度
add_df_to_excel(merged_df, 'author_out')


/var/folders/mg/y652zvws7fnb1d0m_y49f_mw0000gn/T/ipykernel_45734/797925376.py:15: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version
  writer.book = book
/var/folders/mg/y652zvws7fnb1d0m_y49f_mw0000gn/T/ipykernel_45734/797925376.py:21: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [78]:
# 找出每个文件贡献度最高的作者
idx = merged_df.groupby(['file'])['author_commit_percentage'].idxmax()
top_contributors_per_file = merged_df.loc[idx]

# 找出每个文件贡献度最高的作者
add_df_to_excel(top_contributors_per_file, 'top_contributors_per_file')

/var/folders/mg/y652zvws7fnb1d0m_y49f_mw0000gn/T/ipykernel_45734/797925376.py:15: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version
  writer.book = book
/var/folders/mg/y652zvws7fnb1d0m_y49f_mw0000gn/T/ipykernel_45734/797925376.py:21: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [79]:
# 计算每个文件中每个作者的提交次数的比例
merged_df['author_commit_ratio'] = merged_df['commit_count_per_author'] / merged_df['total_commit_count']

# 计算每个文件的熵
file_entropy = merged_df.groupby('file').apply(lambda x: -np.sum(x['author_commit_ratio'] * np.log2(x['author_commit_ratio']))).reset_index(name='entropy')

# 计算每个文件的熵
add_df_to_excel(file_entropy, 'file_entropy')

/var/folders/mg/y652zvws7fnb1d0m_y49f_mw0000gn/T/ipykernel_45734/797925376.py:15: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version
  writer.book = book
/var/folders/mg/y652zvws7fnb1d0m_y49f_mw0000gn/T/ipykernel_45734/797925376.py:21: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [80]:
df['commit_time'] = pd.to_datetime(df['commit_time'])

# Group by file and time period
df['commit_month'] = df['commit_time'].dt.to_period('M')
grouped_df = df.groupby(['file', 'commit_month'])

# Define helper functions
def get_new_devs(group: pd.DataFrame, devs: List[str] = []) -> int:
    unique_devs = group['author'].unique()
    new_devs = len([dev for dev in unique_devs if dev not in devs])
    devs.extend(unique_devs)
    return new_devs

def get_left_devs(group: pd.DataFrame, devs: List[str] = []) -> int:
    unique_devs = group['author'].unique()
    left_devs = len([dev for dev in devs if dev not in unique_devs])
    devs = unique_devs
    return left_devs

# Calculate new and left developers for each time period and file
new_devs = grouped_df.apply(get_new_devs)
left_devs = grouped_df.apply(get_left_devs)

# Combine the results into a new DataFrame
stability_df = pd.DataFrame({
    'file': new_devs.index.get_level_values(0),
    'commit_month': new_devs.index.get_level_values(1),
    'new_developers': new_devs,
    'left_devs': left_devs,
    'stability': new_devs + left_devs
})

# 计算每个文件的稳定性
add_df_to_excel(stability_df, 'stability')

/var/folders/mg/y652zvws7fnb1d0m_y49f_mw0000gn/T/ipykernel_45734/1472527061.py:4: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['commit_month'] = df['commit_time'].dt.to_period('M')
/var/folders/mg/y652zvws7fnb1d0m_y49f_mw0000gn/T/ipykernel_45734/797925376.py:15: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version
  writer.book = book
/var/folders/mg/y652zvws7fnb1d0m_y49f_mw0000gn/T/ipykernel_45734/797925376.py:21: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
